In [ ]:
import os
import librosa
import numpy as np
from random import sample

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# MFCC

In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf
import shutil
from random import sample

# Define number of files for all sets
train_ratio = 0.65  # 65% of all sound should be in the train set
val_ratio = 0.25    # 25% validation set
test_ratio = 0.1    # 10% test set
max_files_per_class = 65  # Maximum number of files per class

basePath = '/content/drive/MyDrive/DL/data/'
imPath = '/content/drive/MyDrive/DL/mfcc/'
destPath = '/content/drive/MyDrive/DL/split_data/mfcc'

birds = []
singleBirdList = []
allFilesList = []

# Get the list of bird directories
for root, dirs, files in os.walk(basePath):
    if root == basePath:
        birds = dirs
print(birds)



['Pitta sordida', 'Dryocopus javensis', 'Caprimulgus macrurus', 'Pnoepyga pusilla', 'Anthipes solitaris', 'Buceros rhinoceros', 'Garulax bicolor']


In [ ]:
trainSet = []
valSet = []
testSet = []

birdsShort = []  # list of short file names
birdNumber = 0

for nr, bird in enumerate(birds):
    for root, dirs, files in os.walk(basePath + bird):
        for file in files:
            if file.endswith(".wav"):
                singleBirdList.append(os.path.join(root, file))

    # Limit to a maximum of 65 files per class
    if len(singleBirdList) > max_files_per_class:
        singleBirdList = sample(singleBirdList, max_files_per_class)

    if len(singleBirdList) > 0:
        birdsShort.append(str(birdNumber) + bird[:5])
        birdNumber += 1
        print("Found ", len(singleBirdList), ' wav files for ', bird)

        # Calculate the number of files for each set
        total_files = len(singleBirdList)
        train_count = int(train_ratio * total_files)
        val_count = int(val_ratio * total_files)
        test_count = total_files - (train_count + val_count)  # Ensure all files are accounted for

        trainSet.append(train_count)
        valSet.append(val_count)
        testSet.append(test_count)

        print("Size of train: ", train_count, ", val: ", val_count, ", test: ", test_count)
        allFilesList.append(singleBirdList)

    singleBirdList = []

print(trainSet)
print(valSet)
print(testSet)

Found  65  wav files for  Pitta sordida
Size of train:  42 , val:  16 , test:  7
Found  65  wav files for  Dryocopus javensis
Size of train:  42 , val:  16 , test:  7
Found  65  wav files for  Caprimulgus macrurus
Size of train:  42 , val:  16 , test:  7
Found  65  wav files for  Pnoepyga pusilla
Size of train:  42 , val:  16 , test:  7
Found  65  wav files for  Anthipes solitaris
Size of train:  42 , val:  16 , test:  7
Found  65  wav files for  Buceros rhinoceros
Size of train:  42 , val:  16 , test:  7
[42, 42, 42, 42, 42, 42]
[16, 16, 16, 16, 16, 16]
[7, 7, 7, 7, 7, 7]


In [ ]:
# Preprocessing function
def preprocess_audio(file_path):
    # Load audio file
    audio, sr = librosa.load(file_path, sr=None)

    # Normalize audio
    audio = audio / np.max(np.abs(audio))  # Normalisasi amplitudo

    # Remove silent parts
    non_silent_indices = librosa.effects.split(audio, top_db=20)  # Menghapus bagian yang tidak bersuara
    audio = np.concatenate([audio[start:end] for start, end in non_silent_indices])

    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)

    return audio, mfccs

In [ ]:
# Randomly choose wav files for each set
trainFiles = []
valFiles = []
testFiles = []

for index, singleBirdList in enumerate(allFilesList):
    randFiles = sample(range(len(singleBirdList)), len(singleBirdList))
    start = 0
    end = trainSet[index]
    trainFiles.append(randFiles[start:end])
    start = end
    end = start + valSet[index]
    valFiles.append(randFiles[start:end])
    start = end
    end = start + testSet[index]
    testFiles.append(randFiles[start:end])
    print("Selected random files number:\n train: ", len(trainFiles[index]), "/", trainSet[index],
          ", val: ", len(valFiles[index]), "/", valSet[index],
          ", test: ", len(testFiles[index]), "/", testSet[index])

Selected random files number:
 train:  42 / 42 , val:  16 / 16 , test:  7 / 7
Selected random files number:
 train:  42 / 42 , val:  16 / 16 , test:  7 / 7
Selected random files number:
 train:  42 / 42 , val:  16 / 16 , test:  7 / 7
Selected random files number:
 train:  42 / 42 , val:  16 / 16 , test:  7 / 7
Selected random files number:
 train:  42 / 42 , val:  16 / 16 , test:  7 / 7
Selected random files number:
 train:  42 / 42 , val:  16 / 16 , test:  7 / 7


In [ ]:
# Function to extract name
def extractName(string):
    return string.rsplit('/', 1)[1].replace(' ', '')[:-4]

# Sort all the lists to make copying files easier
sets = [trainFiles, valFiles, testFiles]
for fileSet in sets:
    for index, files in enumerate(fileSet):
        fileSet[index].sort()

# Change full names to short
setNames = ["train/", "val/", "test/"]

print("Long: ", birds, "\nShort: ", birdsShort)

counter = 0

# Loop untuk setiap jenis burung
for birdNumber, bird in enumerate(birdsShort):  # Menggunakan birdsShort untuk iterasi
    print(f"Processing bird: {bird}")  # Menampilkan burung yang sedang diproses
    counter = 0

    # Periksa untuk semua dataset: train, val, dan test sets
    for setName, fileSet in zip(setNames, sets):
        for setNumber in fileSet[birdNumber]:
            for fileNumber, file in enumerate(allFilesList[birdNumber]):
                if setNumber == fileNumber:  # Jika nomor file yang akan disalin sama dengan nomor file, maka salin
                    # Proses audio menggunakan fungsi preprocess_audio
                    audio, mfccs = preprocess_audio(file)  # Memanggil fungsi preprocessing dan mengassign nilai yang dikembalikan ke audio dan mfccs

                    # **Create the directory if it doesn't exist before writing to it.**
                    destination = os.path.join(destPath, setName, bird)
                    if not os.path.exists(destination):
                        os.makedirs(destination)

                    # Menyimpan file audio yang telah diproses
                    processed_audio_path = os.path.join(destPath, setName, bird, f"{extractName(file)}_processed.wav")
                    sf.write(processed_audio_path, audio, 22050)  # Menyimpan audio yang telah dinormalisasi dan dihapus bagian sunyi

                    # Menyimpan fitur MFCC
                    mfccs_path = os.path.join(destPath, setName, bird, f"{extractName(file)}_mfcc.npy")
                    np.save(mfccs_path, mfccs)  # Menyimpan fitur MFCC sebagai file .npy

                    # Menyalin file terkait dari imPath
                    for root, dirs, files in os.walk(imPath):
                        for file2 in files:
                            if extractName(file) in file2:
                                counter += 1
                                source = os.path.join(root, file2)

                                shutil.copy2(source, destination)

# Menampilkan jumlah file yang telah disalin
print("Total files copied:", counter)

Long:  ['Pitta sordida', 'Dryocopus javensis', 'Caprimulgus macrurus', 'Pnoepyga pusilla', 'Anthipes solitaris', 'Buceros rhinoceros', 'Garulax bicolor'] 
Short:  ['0Pitta', '1Dryoc', '2Capri', '3Pnoep', '4Anthi', '5Bucer']
Processing bird: 0Pitta
Processing bird: 1Dryoc
Processing bird: 2Capri


<ipython-input-4-21613874241c>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing bird: 3Pnoep
Processing bird: 4Anthi
Processing bird: 5Bucer
Total files copied: 513


# MELSPEC

In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf
import shutil
from random import sample
import matplotlib.pyplot as plt

# Define number of files for all sets
train = 0.65  # 65% of all sound should be in the train set
val = 0.25    # 25% validation set
test = 0.1   # 10% test set
basePath = '/content/drive/MyDrive/DL/data/'
imPath = '/content/drive/MyDrive/DL/mels'
destPath = '/content/drive/MyDrive/DL/split_data/melspec'

birds = []
singleBirdList = []
allFilesList = []

# Get list of birds
for root, dirs, files in os.walk(basePath):
    if root == basePath:
        birds = dirs
print(birds)

trainSet = []
valSet = []
testSet = []
birdsShort = []  # list of short file names
birdNumber = 0

# Collect files for each bird
for nr, bird in enumerate(birds):
    for root, dirs, files in os.walk(basePath + bird):
        for file in files:
            if file.endswith(".wav"):
                singleBirdList.append(os.path.join(root, file))
    if len(singleBirdList) > 50:
        birdsShort.append(str(birdNumber) + bird[:5])
        birdNumber += 1
        print("Found ", len(singleBirdList), ' wav files for ', bird)

        # Calculate number of files for each set
        total_files = min(len(singleBirdList), 65)  # Limit to 65 files
        train_count = int(train * total_files)
        val_count = int(val * total_files)
        test_count = total_files - train_count - val_count  # Ensure all files are accounted for

        trainSet.append(train_count)
        valSet.append(val_count)
        testSet.append(test_count)

        print("Size of train: ", train_count, ", val: ", val_count, ", test: ", test_count)
        allFilesList.append(singleBirdList[:total_files])  # Only keep up to 70 files
    singleBirdList = []

print(trainSet)
print(valSet)
print(testSet)

['Pitta sordida', 'Dryocopus javensis', 'Caprimulgus macrurus', 'Pnoepyga pusilla', 'Anthipes solitaris', 'Buceros rhinoceros', 'Garulax bicolor']
Found  178  wav files for  Pitta sordida
Size of train:  42 , val:  16 , test:  7
Found  161  wav files for  Dryocopus javensis
Size of train:  42 , val:  16 , test:  7
Found  200  wav files for  Caprimulgus macrurus
Size of train:  42 , val:  16 , test:  7
Found  200  wav files for  Pnoepyga pusilla
Size of train:  42 , val:  16 , test:  7
Found  65  wav files for  Anthipes solitaris
Size of train:  42 , val:  16 , test:  7
Found  91  wav files for  Buceros rhinoceros
Size of train:  42 , val:  16 , test:  7
[42, 42, 42, 42, 42, 42]
[16, 16, 16, 16, 16, 16]
[7, 7, 7, 7, 7, 7]


In [ ]:
# Preprocessing function
def preprocess_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    audio = audio / np.max(np.abs(audio))  # Normalize audio
    non_silent_indices = librosa.effects.split(audio, top_db=20)
    audio = np.concatenate([audio[start:end] for start, end in non_silent_indices])
    return audio, sr

# Function to create and save mel spectrogram
def save_mel_spectrogram(audio, sr, file_path):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Save as image
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spectrogram_db, sr=sr, x_axis='time', y_axis='mel', fmax=8000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.tight_layout()
    plt.savefig(file_path)
    plt.close()

# Randomly choose wav files for each set
trainFiles = []
valFiles = []
testFiles = []

In [ ]:
for index, singleBirdList in enumerate(allFilesList):
    randFiles = sample(range(len(singleBirdList)), len(singleBirdList))
    start = 0
    end = trainSet[index]
    trainFiles.append(randFiles[start:end])
    start = end
    end = start + valSet[index]
    valFiles.append(randFiles[start:end])
    start = end
    end = start + testSet[index]
    testFiles.append(randFiles[start:end])
    print("Selected random files number:\n train: ", len(trainFiles[index]), "/", trainSet[index],
          "val: ", len(valFiles[index]), "/", valSet[index],
           "test: " , len(testFiles[index]), "/", testSet[index])

# Function to extract name
def extractName(string):
    return string.rsplit('/', 1)[1].replace(' ', '')[:-4]

# Sort all the lists to make copying files easier
sets = [trainFiles, valFiles, testFiles]
for fileSet in sets:
    for index, files in enumerate(fileSet):
        fileSet[index].sort()

# Change full names to short
setNames = ["train/", "val/", "test/"]

print("Long: ", birds, "\nShort: ", birdsShort)





Selected random files number:
 train:  42 / 42 val:  16 / 16 test:  7 / 7
Selected random files number:
 train:  42 / 42 val:  16 / 16 test:  7 / 7
Selected random files number:
 train:  42 / 42 val:  16 / 16 test:  7 / 7
Selected random files number:
 train:  42 / 42 val:  16 / 16 test:  7 / 7
Selected random files number:
 train:  42 / 42 val:  16 / 16 test:  7 / 7
Selected random files number:
 train:  42 / 42 val:  16 / 16 test:  7 / 7
Long:  ['Pitta sordida', 'Dryocopus javensis', 'Caprimulgus macrurus', 'Pnoepyga pusilla', 'Anthipes solitaris', 'Buceros rhinoceros', 'Garulax bicolor'] 
Short:  ['0Pitta', '1Dryoc', '2Capri', '3Pnoep', '4Anthi', '5Bucer']


In [ ]:
counter = 0
# Loop untuk setiap jenis burung
for birdNumber, bird in enumerate(birdsShort):  # Menggunakan birdsShort untuk iterasi
    print(f"Processing bird: {bird}")  # Menampilkan burung yang sedang diproses
    counter = 0

    # Periksa untuk semua dataset: train, val, dan test sets
    for setName, fileSet in zip(setNames, sets):
        for setNumber in fileSet[birdNumber]:
            for fileNumber, file in enumerate(allFilesList[birdNumber]):
                if setNumber == fileNumber:  # Jika nomor file yang akan disalin sama dengan nomor file, maka salin
                    # Proses audio menggunakan fungsi preprocess_audio
                    audio, sr = preprocess_audio(file)  # Memanggil fungsi preprocessing dan menyimpan hasilnya

                    # **Create the directory if it doesn't exist before writing to it.**
                    destination = os.path.join(destPath, setName, bird)
                    if not os.path.exists(destination):
                        os.makedirs(destination)

                    # Menyimpan mel spectrogram
                    mel_spectrogram_path = os.path.join(destination, f"{extractName(file)}_mel_spectrogram.png")
                    save_mel_spectrogram(audio, sr, mel_spectrogram_path)  # Menyimpan mel spectrogram sebagai gambar

                    # Menyalin file terkait dari imPath
                    for root, dirs, files in os.walk(imPath):
                        for file2 in files:
                            if extractName(file) in file2:
                                counter += 1
                                source = os.path.join(root, file2)

                                shutil.copy2(source, destination)  # Menyalin file terkait ke direktori tujuan

# Menampilkan jumlah file yang telah disalin
print("Total files copied:", counter)

Processing bird: 0Pitta
Processing bird: 1Dryoc
Processing bird: 2Capri
Processing bird: 3Pnoep
Processing bird: 4Anthi
Processing bird: 5Bucer
Total files copied: 633
